In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl
import h5py
import tqdm
import os


from holodeck import plot, detstats
import holodeck.single_sources as sings
from holodeck.constants import YR, MSOL, MPC, GYR, PC
import holodeck as holo
from holodeck.sams import sam

import hasasia.sim as hsim

import sys
sys.path.append('/Users/emigardiner/GWs/holodeck/ecg-notebooks/parameter_investigation')
import anatomy as anat

In [ ]:
CALC = False

In [ ]:
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    '''
    https://stackoverflow.com/a/18926541
    '''
    if isinstance(cmap, str):
        cmap = plt.get_cmap(cmap)
    new_cmap = mpl.colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

cmap_base = 'magma_r'
magma_r = truncate_colormap(cmap_base, 0, 0.85)
blacks = truncate_colormap('binary', 0.4, 1.0)

In [ ]:
SHAPE = None
NREALS = 500
# NREALS = 20
NFREQS = 40
NLOUDEST = 10

BUILD_ARRAYS = False
SAVEFIG = False
TOL=0.01
MAXBADS=5

NVARS = 3
# NVARS = 6

NPSRS = 40
NSKIES = 25
# NSKIES = 15
RED_GAMMA = None
RED2WHITE = None



In [ ]:
def get_var_data( target, var=int(NVARS/2), nvars=NVARS, nreals=NREALS, nskies=NSKIES, shape=SHAPE, red_gamma = None, red2white=None,
    path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz'     
):

    path = path + f'/{target}_v{nvars}_r{nreals}_shape{str(shape)}' 
    load_data_from_file = path+f'/data_params.npz' 
    load_dets_from_file = path+f'/detstats_s{nskies}' 
    if red_gamma is not None and red2white is not None:
        load_dets_from_file = load_dets_from_file+f'_r2w{red2white:.1f}_rg{red_gamma:.1f}'
    else:
        load_dets_from_file = load_dets_from_file+f'_white'
    load_dets_from_file = load_dets_from_file+'.npz'

    if os.path.exists(load_data_from_file) is False:
        err = f"load data file '{load_data_from_file}' does not exist, you need to construct it."
        raise Exception(err)
    if os.path.exists(load_dets_from_file) is False:
        err = f"load dets file '{load_dets_from_file}' does not exist, you need to construct it."
        raise Exception(err)
    file = np.load(load_data_from_file, allow_pickle=True)
    data = file['data'][var]
    params = file['params'][var]
    file.close()

    file = np.load(load_dets_from_file, allow_pickle=True)
    dsdat = file['dsdat'][var]
    file.close()

    return data, params, dsdat

In [ ]:
data, params, dsdat = get_var_data('gsmf_phi0', var=1)

In [ ]:
print(dsdat['gamma_ssi'].shape)
print(data['sspar'].shape)

In [ ]:
ssmtt = data['sspar'][0]/MSOL
ssmtt = np.repeat(ssmtt, NSKIES).reshape(NFREQS, NREALS, NLOUDEST, NSKIES)
ssmtt = np.swapaxes(ssmtt, -2, -1)

dpssi = dsdat['gamma_ssi']

Get edges

In [ ]:
NBINS = 40
sam = holo.sams.Semi_Analytic_Model()
mt_edges = sam.mtot/MSOL
# ff_edges = data['fobs_edges']*YR

dpmin = np.min(dpssi)
dpmax = np.max(dpssi)
print(dpmin, dpmax)
dp_edges = np.geomspace(2.5e-6,1.0, NBINS)

In [ ]:
hist, mme, ffe = np.histogram2d(
    dpssi.flatten(), ssmtt.flatten(), bins=(dp_edges, mt_edges))
mtgrid, dpgrid, = np.meshgrid(mt_edges, dp_edges)

In [ ]:
snssi = dsdat['snr_ss']
snmin = np.min(snssi)
snmax = np.max(snssi)
print(snmin, snmax)
sn_edges = np.geomspace(2.e-6,25, NBINS)

In [ ]:
TARGET = 'gsmf_phi0'
NSKIES = 25

# get edges
NBINS = 40
sam = holo.sams.Semi_Analytic_Model()
mt_edges = sam.mtot[15:-10]/MSOL
sn_edges = np.geomspace(2.e-6,25, NBINS)

# build grid from edges
mtgrid, sngrid, = np.meshgrid(mt_edges, sn_edges)

xlabel='Mass [M$_\odot$]'
ylabel='SNR'

fig, axs = plot.figax_single(
    ncols=3, sharex=True, sharey=True, height=3.5)


for ii in [0,1,2]: # vars to use
    data, params, dsdat = get_var_data(target=TARGET, var=ii, nskies=NSKIES)
    ssmtt = data['sspar'][0]/MSOL
    ssmtt = np.repeat(ssmtt, NSKIES).reshape(NFREQS, NREALS, NLOUDEST, NSKIES)
    ssmtt = np.swapaxes(ssmtt, -2, -1)

    snssi = dsdat['snr_ss']

    # get histogram
    hist, mte, sne = np.histogram2d(
        snssi.flatten(), ssmtt.flatten(), bins=(sn_edges, mt_edges))

    im = axs[ii].pcolormesh(mtgrid, sngrid, np.log10(hist), cmap=cm.BuPu)
    title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    axs[ii].text(0, 1, title, horizontalalignment='left', verticalalignment='top', 
                 transform=axs[ii].transAxes, color='white')

# fig.colorbar(im, cax=axs[-1], label='$\log N$', orientation='vertical')
axs[0].set_ylabel(ylabel)
axs[1].set_xlabel(xlabel)
fig.tight_layout()
plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.4, 
             orientation='horizontal', pad=0.22, label='$\log N$', )


In [ ]:
TARGET = 'gsmf_phi0'
NSKIES = 25

# get edges
NBINS = 40
sam = holo.sams.Semi_Analytic_Model()
mt_edges = sam.mtot[25:-12]/MSOL
sn_edges = np.geomspace(2.e-6,25, NBINS)

# build grid from edges
mtgrid, sngrid, = np.meshgrid(mt_edges, sn_edges)

xlabel='Mass [M$_\odot$]'
ylabel='SNR'

fig, axs = plot.figax_single(
    nrows=3, sharex=True, sharey=True, height=11)

for ii in [0,1,2]: # vars to use
    data, params, dsdat = get_var_data(target=TARGET, var=ii, nskies=NSKIES)
    ssmtt = data['sspar'][0]/MSOL
    ssmtt = np.repeat(ssmtt, NSKIES).reshape(NFREQS, NREALS, NLOUDEST, NSKIES)
    ssmtt = np.swapaxes(ssmtt, -2, -1)

    snssi = dsdat['snr_ss']

    # get histogram
    hist, mte, sne = np.histogram2d(
        snssi.flatten(), ssmtt.flatten(), bins=(sn_edges, mt_edges))

    im = axs[ii].pcolormesh(mtgrid, sngrid, np.log10(hist))
    title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    axs[ii].text(0, 1, title, horizontalalignment='left', verticalalignment='top', 
                 transform=axs[ii].transAxes, color='white', fontsize=18)
    axs[ii].set_facecolor('k')

axs[1].set_ylabel(ylabel)
axs[2].set_xlabel(xlabel)
fig.tight_layout()
plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.7, 
             orientation='horizontal', pad=0.08, label='$\log N$', )


In [ ]:
TARGET = 'gsmf_phi0'

# get edges
NBINS = 40
sam = holo.sams.Semi_Analytic_Model()
mt_edges = sam.mtot[25:-15]/MSOL
sn_edges = np.geomspace(2.e-6,25, NBINS)

# build grid from edges
mtgrid, sngrid, = np.meshgrid(mt_edges, sn_edges)

# titles
xlabel='Mass [M$_\odot$]'
ylabel='SNR'

# colormaps
# cmaps = [cm.Greens, cm.Blues, cm.Purples]
cmaps = [cm.Oranges, cm.Greys, cm.Blues]


fig, ax = plot.figax_single()

for ii in [0,1,2]: # vars to use
    data, params, dsdat = get_var_data(target=TARGET, var=ii)
    ssmtt = data['sspar'][0]/MSOL
    ssmtt = np.repeat(ssmtt, NSKIES).reshape(NFREQS, NREALS, NLOUDEST, NSKIES)
    ssmtt = np.swapaxes(ssmtt, -2, -1)

    snssi = dsdat['snr_ss']

    # get histogram
    hist, mte, sne = np.histogram2d(
        snssi.flatten(), ssmtt.flatten(), bins=(sn_edges, mt_edges))

    im = ax.pcolormesh(mtgrid, sngrid, np.log10(hist), cmap=cmaps[ii], alpha=0.5)
    title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    # axs[ii].text(0, 1, title, horizontalalignment='left', verticalalignment='top', 
    #              transform=axs[ii].transAxes, color='white')

ax.set_ylabel(ylabel)
ax.set_xlabel(xlabel)

fig.tight_layout()

In [ ]:
TARGET = 'hard_time'
NSKIES = 50


# get edges
NBINS = 40
sam = holo.sams.Semi_Analytic_Model()
mt_edges = sam.mtot[20:-12]/MSOL
sn_edges = np.geomspace(2.e-6,515, NBINS)

# build grid from edges
mtgrid, sngrid, = np.meshgrid(mt_edges, sn_edges)

xlabel='Mass [M$_\odot$]'
ylabel='SNR'

fig, axs = plot.figax_single(
    nrows=3, sharex=True, sharey=True, height=11)

for ii in [0,1,2]: # vars to use
    data, params, dsdat = get_var_data(target=TARGET, var=ii, nskies=NSKIES)
    ssmtt = data['sspar'][0]/MSOL
    ssmtt = np.repeat(ssmtt, NSKIES).reshape(NFREQS, NREALS, NLOUDEST, NSKIES)
    ssmtt = np.swapaxes(ssmtt, -2, -1)

    snssi = dsdat['snr_ss']

    # get histogram
    hist, mte, sne = np.histogram2d(
        snssi.flatten(), ssmtt.flatten(), bins=(sn_edges, mt_edges))

    im = axs[ii].pcolormesh(mtgrid, sngrid, np.log10(hist), cmap=cm.magma)
    title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    axs[ii].text(0, 1, title, horizontalalignment='left', verticalalignment='top', 
                 transform=axs[ii].transAxes, color='white', fontsize=18)
    # axs[ii].set_facecolor('k')

axs[1].set_ylabel(ylabel)
axs[2].set_xlabel(xlabel)
fig.tight_layout()
plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.7, 
             orientation='horizontal', pad=0.08, label='$\log N$', )


In [ ]:
TARGET = 'hard_time'
NSKIES = 50

# get edges
NBINS = 40
sam = holo.sams.Semi_Analytic_Model()
mt_edges = sam.mtot[25:-15]/MSOL
sn_edges = np.geomspace(2.e-6,25, NBINS)

# build grid from edges
mtgrid, sngrid, = np.meshgrid(mt_edges, sn_edges)

# titles
xlabel='Mass [M$_\odot$]'
ylabel='SNR'

# colormaps
# cmaps = [cm.Greens, cm.Blues, cm.Purples]
cmaps = [cm.Oranges, cm.Greys, cm.Blues]


fig, ax = plot.figax_single()

for ii in [0,1,2]: # vars to use
    data, params, dsdat = get_var_data(target=TARGET, var=ii, nskies=NSKIES)
    ssmtt = data['sspar'][0]/MSOL
    ssmtt = np.repeat(ssmtt, NSKIES).reshape(NFREQS, NREALS, NLOUDEST, NSKIES)
    ssmtt = np.swapaxes(ssmtt, -2, -1)

    snssi = dsdat['snr_ss']

    # get histogram
    hist, mte, sne = np.histogram2d(
        snssi.flatten(), ssmtt.flatten(), bins=(sn_edges, mt_edges))

    im = ax.pcolormesh(mtgrid, sngrid, np.log10(hist), cmap=cmaps[ii], alpha=0.5)
    title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    # axs[ii].text(0, 1, title, horizontalalignment='left', verticalalignment='top', 
    #              transform=axs[ii].transAxes, color='white')

ax.set_ylabel(ylabel)
ax.set_xlabel(xlabel)

fig.tight_layout()

# Hard_time 5ax

In [ ]:
TARGET = 'hard_time'
NSKIES = 100
NVARS = 21

def calculate_histograms(TARGET, NFREQS, NREALS, NLOUDEST, NSKIES):
    # get edges
    NBINS = 40
    sam = holo.sams.Semi_Analytic_Model()
    mt_edges = sam.mtot[20:-12]/MSOL
    sn_edges = np.geomspace(2.e-6,515, NBINS)
    dc_edges = np.geomspace(1e1, 1e4, NBINS)
    ff_edges = None # assign in loops

    # get histograms
    hist_mt = []
    hist_dc = []
    hist_ff = []
    bghist_mt = []
    bghist_dc = []
    bghist_ff = []
    text = []
    for ii in [0,5,10,15,20]: # vars to use
        data, params, dsdat = get_var_data(target=TARGET, var=ii, nskies=NSKIES, nvars=NVARS,
                        path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz'  )
        sspar = sings.all_sspars(data['fobs_cents'], data['sspar'])
        bgpar = data['bgpar']
        ff_edges = data['fobs_edges']*YR

        # total mass
        _ssmtt = sspar[0]/MSOL
        _ssmtt = np.repeat(_ssmtt, NSKIES).reshape(NFREQS, NREALS, NLOUDEST, NSKIES)
        _ssmtt = np.swapaxes(_ssmtt, -2, -1)

        _bgmtt = bgpar[0]/MSOL # F,R
        print(f"{_bgmtt.shape=}")

        # comoving distance
        _ssdcm = sspar[4]/MPC
        _ssdcm = np.repeat(_ssdcm, NSKIES).reshape(NFREQS, NREALS, NLOUDEST, NSKIES)
        _ssdcm = np.swapaxes(_ssdcm, -2, -1)

        _bgdcm = bgpar[4]/MPC

        # frequencies
        _bgfrq = data['fobs_cents']*YR
        _bgfrq = np.repeat(_bgfrq, NREALS).reshape(NFREQS, NREALS)
        _ssfrq = np.repeat(_bgfrq, NSKIES*NLOUDEST).reshape(NFREQS, NREALS, NLOUDEST, NSKIES)

        # snr
        _snssi = dsdat['snr_ss']
        _snrbg = dsdat['snr_bg']
        _snrbg = np.repeat(_snrbg, NFREQS).reshape(NREALS, NFREQS)
        _snrbg = np.swapaxes(_snrbg, 0, 1)
        print(f"{_snrbg.shape=}")

        # get total mass histograms
        _hist_mt, sne, mte, = np.histogram2d(
            _snssi.flatten(), _ssmtt.flatten(), bins=(sn_edges, mt_edges))
        hist_mt.append(_hist_mt)

        _bghist_mt, sne, mte, = np.histogram2d(
            _snrbg.flatten(), _bgmtt.flatten(), bins=(sn_edges, mt_edges))
        bghist_mt.append(_bghist_mt)

        # get comoving distance histograms
        _hist_dc, sne, dce, = np.histogram2d(
            _snssi.flatten(), _ssdcm.flatten(), bins=(sn_edges, dc_edges))
        hist_dc.append(_hist_dc)

        _bghist_dc, sne, dce = np.histogram2d(
            _snrbg.flatten(), _bgdcm.flatten(), bins=(sn_edges, dc_edges))
        bghist_dc.append(_bghist_dc)

        # get frequencie histograms
        _hist_ff, sne, fqe, = np.histogram2d(
            _snssi.flatten(), _ssfrq.flatten(), bins=(sn_edges, ff_edges))
        hist_ff.append(_hist_ff)

        _bghist_ff, sne, fqe = np.histogram2d(
            _snrbg.flatten(), _bgfrq.flatten(), bins=(sn_edges, ff_edges))
        bghist_ff.append(_bghist_ff)
        

        _text = '%.2f' % (params[TARGET])
        text.append(_text)
    np.savez('/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata'
            +f'/snr_hist_{TARGET}_v{NVARS}_5axs.npz',
            mt_edges=mt_edges, sn_edges=sn_edges, dc_edges=dc_edges,
            hist_mt=hist_mt, hist_dc=hist_dc, hist_ff=hist_ff, text=text,
            bghist_mt = bghist_mt, bghist_dc=bghist_dc, bghist_ff=bghist_ff,
            )


if CALC: 
    calculate_histograms(TARGET, NFREQS, NREALS, NLOUDEST, NSKIES)
else:
    file = np.load('/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata'
            +f'/snr_hist_{TARGET}_v{NVARS}_5axs.npz')
    mt_edges = file['mt_edges']
    sn_edges = file['sn_edges']
    dc_edges = file['dc_edges']
    # ssmtt = file['ssmtt']
    # ssdcm = file['ssdcm']
    # snssi = file['snssi']
    hist_mt = file['hist_mt']
    hist_dc = file['hist_dc']
    hist_ff = file['hist_ff']
    text = file['text']
    bghist_mt = file['bghist_mt']
    bghist_dc = file['bghist_dc']
    bghist_ff = file['bghist_ff']
    file.close()

In [ ]:
# PLOT
xlabel='Mass [M$_\odot$]'
ylabel='SNR'

# build grid from edges
mtgrid, sngrid, = np.meshgrid(mt_edges, sn_edges)

# make figure
fig, axs = plot.figax_double(
    ncols=5, sharex=True, sharey=True, height=4)

# plot histograms
for aa, ax in enumerate(axs):
    im = ax.pcolormesh(mtgrid, sngrid, np.log10(hist_mt[aa]), cmap=magma_r)
    # title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    ax.text(0.01, 0.99, '$\\tau_\mathrm{hard}$=%s' % text[aa], horizontalalignment='left', verticalalignment='top', 
                 transform=axs[aa].transAxes, color='k', fontsize=12)
    # axs[ii].set_facecolor('k')

# set labels
axs[0].set_ylabel(ylabel)
axs[2].set_xlabel(xlabel)
fig.tight_layout()

plt.subplots_adjust(wspace=0)

# add colorbar
plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.7, 
             orientation='vertical', pad=0.01, label='$\log N$', )

# save
fig.savefig('/Users/emigardiner/GWs/holodeck/output/figures/bigplots/snr_hist'
            + f'/snr_vs_mass_hist_{TARGET}_v{NVARS}_5axs.png', dpi=100)

In [ ]:
# PLOT
xlabel='Distance [Mpc]'
ylabel='SNR'

# build grid from edges
dcgrid, sngrid, = np.meshgrid(dc_edges, sn_edges)

# make figure
fig, axs = plot.figax_double(
    ncols=5, sharex=True, sharey=True, height=4)

# plot histograms
for aa, ax in enumerate(axs):
    im = ax.pcolormesh(dcgrid, sngrid, np.log10(hist_dc[aa]), cmap=cm.viridis_r)
    # title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    ax.text(0.01, 0.99, '$\\tau_\mathrm{hard}$=%s' % text[aa], horizontalalignment='left', verticalalignment='top', 
                 transform=axs[aa].transAxes, color='k', fontsize=12)
    # axs[ii].set_facecolor('k')

# set labels
axs[0].set_ylabel(ylabel)
axs[2].set_xlabel(xlabel)
fig.tight_layout()

plt.subplots_adjust(wspace=0)

# add colorbar
plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.7, 
             orientation='vertical', pad=0.01, label='$\log N$', )

# save
fig.savefig('/Users/emigardiner/GWs/holodeck/output/figures/bigplots/snr_hist'
            + f'/snr_vs_dcom_hist_{TARGET}_v{NVARS}_5axs.png', dpi=100)

In [ ]:
# PLOT
xlabel='Frequency [yr$^{-1}$]'
ylabel='SNR'

# build grid from edges
ffgrid, sngrid, = np.meshgrid(ff_edges, sn_edges)

# make figure
fig, axs = plot.figax_double(
    ncols=5, sharex=True, sharey=True, height=4)

# plot histograms
for aa, ax in enumerate(axs):
    im = ax.pcolormesh(ffgrid, sngrid, np.log10(hist_ff[aa]), cmap=cm.cubehelix_r)
    # title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    ax.text(0.01, 0.99, '$\\tau_\mathrm{hard}$=%s' % text[aa], horizontalalignment='left', verticalalignment='top', 
                 transform=axs[aa].transAxes, color='k', fontsize=12)
    # axs[ii].set_facecolor('k')

# set labels
axs[0].set_ylabel(ylabel)
axs[2].set_xlabel(xlabel)
fig.tight_layout()

plt.subplots_adjust(wspace=0)

# add colorbar
plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.7, 
             orientation='vertical', pad=0.01, label='$\log N$', )

# save
fig.savefig('/Users/emigardiner/GWs/holodeck/output/figures/bigplots/snr_hist'
            + f'/snr_vs_dcom_hist_{TARGET}_v{NVARS}_5axs.png', dpi=100)

Combined

In [ ]:
# PLOT
xlabel1='Mass [M$_\odot$]'
xlabel2='Distance [Mpc]'
xlabel3='Frequency [yr$^{-1}$]'
ylabel='SNR'

# build grid from edges
mtgrid, sngrid1, = np.meshgrid(mt_edges, sn_edges)
dcgrid, sngrid2, = np.meshgrid(dc_edges, sn_edges)
ffgrid, sngrid3 = np.meshgrid(ff_edges, sn_edges)

# make figure
fig, axs = plot.figax_double(
    ncols=5, nrows=3, sharex=False, sharey=True, height=8)

# plot histograms
for aa, ax in enumerate(axs[0,:]):
    im1 = ax.pcolormesh(mtgrid, sngrid1, np.log10(hist_mt[aa]), cmap=magma_r)
    # title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    ax.text(0.01, 0.99, '$\\tau_\mathrm{hard}$=%s' % text[aa], horizontalalignment='left', verticalalignment='top', 
                 transform=ax.transAxes, color='k', fontsize=12)
#     # axs[ii].set_facecolor('k')

# plot dcom histograms
for aa, ax in enumerate(axs[1,:]):
    im2 = ax.pcolormesh(dcgrid, sngrid2, np.log10(hist_dc[aa]), cmap=cm.viridis_r)
    # title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    # ax.text(0.01, 0.99, '$\\tau_\mathrm{hard}$=%s' % text[aa], horizontalalignment='left', verticalalignment='top', 
    #              transform=ax.transAxes, color='k', fontsize=12)

# plot freq histograms
for aa, ax in enumerate(axs[2,:]):
    im3 = ax.pcolormesh(ffgrid, sngrid3, np.log10(hist_ff[aa]), cmap=cm.cubehelix_r)
    # title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    ax.text(0.01, 0.01, '$\\tau_\mathrm{hard}$=%s' % text[aa], horizontalalignment='left', verticalalignment='bottom', 
                 transform=ax.transAxes, color='k', fontsize=12)



# set labels
axs[1,0].set_ylabel(ylabel)
axs[0,2].set_xlabel(xlabel1)
axs[1,2].set_xlabel(xlabel2)
axs[2,2].set_xlabel(xlabel3)
fig.tight_layout()

plt.subplots_adjust(wspace=0)
# add colorbar
for ii, im in enumerate([im1, im2, im3]):
    plt.colorbar(im, ax=axs[ii,:].ravel().tolist(), shrink=0.7, 
                orientation='vertical', pad=0.03, label='$\log N$', )
# save
fig.savefig('/Users/emigardiner/GWs/holodeck/output/figures/bigplots/snr_hist'
            + f'/snr_vs_props_hist_{TARGET}_v{NVARS}_5axs.png', dpi=100)

In [ ]:
# PLOT
xlabel1='Mass [M$_\odot$]'
xlabel2='Distance [Mpc]'
xlabel3='Frequency [yr$^{-1}$]'
ylabel='SNR'

# build grid from edges
mtgrid, sngrid1, = np.meshgrid(mt_edges, sn_edges)
dcgrid, sngrid2, = np.meshgrid(dc_edges, sn_edges)
ffgrid, sngrid3 = np.meshgrid(ff_edges, sn_edges)

# make figure
fig, axs = plot.figax_double(
    ncols=5, nrows=3, sharex=False, sharey=True, height=8)

# plot histograms
for aa, ax in enumerate(axs[0,:]):
    im1 = ax.pcolormesh(mtgrid, sngrid1, np.log10(hist_mt[aa]), cmap=cm.Blues, vmin=0, vmax=5)
    # title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    ax.text(0.01, 0.99, '$\\tau_\mathrm{hard}$=%s' % text[aa], horizontalalignment='left', verticalalignment='top', 
                 transform=ax.transAxes, color='k', fontsize=12)
#     # axs[ii].set_facecolor('k')

# plot dcom histograms
for aa, ax in enumerate(axs[1,:]):
    im2 = ax.pcolormesh(dcgrid, sngrid2, np.log10(hist_dc[aa]), cmap=cm.Blues, vmin=0, vmax=5)
    # title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    # ax.text(0.01, 0.99, '$\\tau_\mathrm{hard}$=%s' % text[aa], horizontalalignment='left', verticalalignment='top', 
    #              transform=ax.transAxes, color='k', fontsize=12)

# plot freq histograms
for aa, ax in enumerate(axs[2,:]):
    im3 = ax.pcolormesh(ffgrid, sngrid3, np.log10(hist_ff[aa]), cmap=cm.Blues, vmin=0, vmax=5)
    # title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    ax.text(0.01, 0.01, '$\\tau_\mathrm{hard}$=%s' % text[aa], horizontalalignment='left', verticalalignment='bottom', 
                 transform=ax.transAxes, color='k', fontsize=12)



# set labels
axs[1,0].set_ylabel(ylabel)
axs[0,2].set_xlabel(xlabel1)
axs[1,2].set_xlabel(xlabel2)
axs[2,2].set_xlabel(xlabel3)
fig.tight_layout()

plt.subplots_adjust(wspace=0)
# add colorbar
# for ii, im in enumerate([im1, im2, im3]):
plt.colorbar(im2, ax=axs.ravel().tolist(), shrink=0.5, 
            orientation='vertical', pad=0.03, label='$\log N$', )
# save
fig.savefig('/Users/emigardiner/GWs/holodeck/output/figures/bigplots/snr_hist'
            + f'/snr_vs_props_bluehist_{TARGET}_v{NVARS}_5axs.png', dpi=100)

In [ ]:
# PLOT
xlabel1='Mass [M$_\odot$]'
xlabel2='Distance [Mpc]'
xlabel3='Frequency [yr$^{-1}$]'
ylabel='SNR'

# build grid from edges
mtgrid, sngrid1, = np.meshgrid(mt_edges, sn_edges)
dcgrid, sngrid2, = np.meshgrid(dc_edges, sn_edges)
ffgrid, sngrid3 = np.meshgrid(ff_edges, sn_edges)

# make figure
fig, axs = plot.figax_single(
    ncols=3, nrows=3, sharex=False, sharey=True, height=8)

cols = [0,2,4]

# plot histograms
for aa, ax in enumerate(axs[0,:]):
    im1 = ax.pcolormesh(mtgrid, sngrid1, np.log10(hist_mt[cols[aa]]), cmap=cm.Blues, vmin=0, vmax=5)
    # title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    ax.text(0.01, 0.99, '$\\tau_\mathrm{hard}$=%s' % text[cols[aa]], horizontalalignment='left', verticalalignment='top', 
                 transform=ax.transAxes, color='k', fontsize=12)
#     # axs[ii].set_facecolor('k')

# plot dcom histograms
for aa, ax in enumerate(axs[1,:]):
    im2 = ax.pcolormesh(dcgrid, sngrid2, np.log10(hist_dc[cols[aa]]), cmap=cm.Blues, vmin=0, vmax=5)
    # title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    # ax.text(0.01, 0.99, '$\\tau_\mathrm{hard}$=%s' % text[cols[aa]], horizontalalignment='left', verticalalignment='top', 
    #              transform=ax.transAxes, color='k', fontsize=12)

# plot freq histograms
for aa, ax in enumerate(axs[2,:]):
    im3 = ax.pcolormesh(ffgrid, sngrid3, np.log10(hist_ff[cols[aa]]), cmap=cm.Blues, vmin=0, vmax=5)
    # title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    ax.text(0.01, 0.01, '$\\tau_\mathrm{hard}$=%s' % text[cols[aa]], horizontalalignment='left', verticalalignment='bottom', 
                 transform=ax.transAxes, color='k', fontsize=12)



# set labels
axs[1,0].set_ylabel(ylabel)
axs[0,1].set_xlabel(xlabel1)
axs[1,1].set_xlabel(xlabel2)
axs[2,1].set_xlabel(xlabel3)
fig.tight_layout()

plt.subplots_adjust(wspace=0)
# add colorbar
# for ii, im in enumerate([im1, im2, im3]):
plt.colorbar(im2, ax=axs.ravel().tolist(), shrink=0.5, 
            orientation='vertical', pad=0.03, label='$\log N$', )
# save
fig.savefig('/Users/emigardiner/GWs/holodeck/output/figures/bigplots'
            + f'/snr_vs_props_bluehist_{TARGET}_v{NVARS}_3axs.png', dpi=100)

### Add background

In [ ]:
def plot_hists_3ax_with_bg(TARGET, NVARS, mt_edges, dc_edges, ff_edges,
    sn_edges, hist_mt, hist_dc, hist_ff, 
    bghist_mt, bghist_dc, bghist_ff,
    ss_cmap, label,
):
    # PLOT
    xlabel1='Mass [M$_\odot$]'
    xlabel2='Distance [Mpc]'
    xlabel3='Frequency [yr$^{-1}$]'
    ylabel='SNR'

    # build grid from edges
    mtgrid, sngrid1, = np.meshgrid(mt_edges, sn_edges)
    dcgrid, sngrid2, = np.meshgrid(dc_edges, sn_edges)
    ffgrid, sngrid3 = np.meshgrid(ff_edges, sn_edges)

    # make figure
    fig, axs = plot.figax_single(
        ncols=3, nrows=3, sharex=False, sharey=True, height=8)

    cols = [0,2,4]

    # plot histograms
    for aa, ax in enumerate(axs[0,:]):
        im1 = ax.pcolormesh(mtgrid, sngrid1, np.log10(hist_mt[cols[aa]]), cmap=ss_cmap, vmin=0, vmax=5)
        im0 = ax.pcolormesh(mtgrid, sngrid1, np.log10(bghist_mt[cols[aa]]), cmap=blacks, alpha=0.75)
        # title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
        ax.text(0.01, 0.99, f"{label}={text[cols[aa]]}", horizontalalignment='left', verticalalignment='top', 
                    transform=ax.transAxes, color='k', fontsize=12)
    #     # axs[ii].set_facecolor('k')

    # plot dcom histograms
    for aa, ax in enumerate(axs[1,:]):
        im2 = ax.pcolormesh(dcgrid, sngrid2, np.log10(hist_dc[cols[aa]]), cmap=ss_cmap, vmin=0, vmax=5)
        im0 = ax.pcolormesh(dcgrid, sngrid2, np.log10(bghist_dc[cols[aa]]), cmap=blacks, alpha=0.75)
        # title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
        # ax.text(0.01, 0.99, '$\\tau_\mathrm{hard}$=%s' % text[cols[aa]], horizontalalignment='left', verticalalignment='top', 
        #              transform=ax.transAxes, color='k', fontsize=12)

    # plot freq histograms
    for aa, ax in enumerate(axs[2,:]):
        im3 = ax.pcolormesh(ffgrid, sngrid3, np.log10(hist_ff[cols[aa]]), cmap=ss_cmap, vmin=0, vmax=5)
        im0 = ax.pcolormesh(ffgrid, sngrid3, np.log10(bghist_ff[cols[aa]]), cmap=blacks, alpha=0.75)
        # title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
        ax.text(0.01, 0.01, f"{label}={text[cols[aa]]}", horizontalalignment='left', verticalalignment='bottom', 
                    transform=ax.transAxes, color='k', fontsize=12)



    # set labels
    axs[1,0].set_ylabel(ylabel)
    axs[0,1].set_xlabel(xlabel1)
    axs[1,1].set_xlabel(xlabel2)
    axs[2,1].set_xlabel(xlabel3)
    fig.tight_layout()

    plt.subplots_adjust(wspace=0)
    # add colorbar
    # for ii, im in enumerate([im1, im2, im3]):
    plt.colorbar(im2, ax=axs.ravel().tolist(), shrink=0.5, 
                orientation='vertical', pad=0.03, label='$\log N$', )
    # save
    fig.savefig('/Users/emigardiner/GWs/holodeck/output/figures/bigplots/snr_hist'
                + f'/snr_vs_props_hist_3axs_bg_{TARGET}_v{NVARS}.png', dpi=100)

fig = plot_hists_3ax_with_bg(
    TARGET, NVARS, mt_edges, dc_edges, ff_edges, sn_edges,
    hist_mt, hist_dc, hist_ff, bghist_mt, bghist_dc, bghist_ff,
    ss_cmap=cm.Blues, label = "$\\tau_\mathrm{hard}$")

In [ ]:
# PLOT
xlabel='Mass [M$_\odot$]'
ylabel='SNR'

# build grid from edges
mtgrid, sngrid, = np.meshgrid(mt_edges, sn_edges)

# make figure
fig, axs = plot.figax_single(
    nrows=5, sharex=True, sharey=True, height=16)

# plot histograms
for aa, ax in enumerate(axs):
    im = axs[aa].pcolormesh(mtgrid, sngrid, np.log10(hist_mt[aa]), cmap=cm.magma_r)
    # title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    axs[aa].text(0, 1, '$\\tau_\mathrm{hard}$=%s' % text[aa], 
                 horizontalalignment='left', verticalalignment='top', 
                 transform=axs[aa].transAxes, color='k', fontsize=18)
    # axs[ii].set_facecolor('k')

# set labels
axs[2].set_ylabel(ylabel)
axs[-1].set_xlabel(xlabel)
fig.tight_layout()
plt.subplots_adjust(hspace=0)

# add colorbar
plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.7, 
             orientation='horizontal', pad=0.08, label='$\log N$', )


# save
fig.savefig('/Users/emigardiner/GWs/holodeck/output/figures/bigplots/snr_hist'
            + f'/snr_vs_mass_numden_{TARGET}_v{NVARS}_5axs.png', dpi=100)

# Hard_gamma_inner 5ax

In [ ]:
TARGET = 'hard_gamma_inner'
NSKIES = 100
NVARS = 21
# CALC=False

if CALC: 
    calculate_histograms(TARGET, NFREQS, NREALS, NLOUDEST, NSKIES)
else:
    file = np.load('/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata'
            +f'/snr_hist_{TARGET}_v{NVARS}_5axs.npz')
    mt_edges = file['mt_edges']
    sn_edges = file['sn_edges']
    dc_edges = file['dc_edges']
    # ssmtt = file['ssmtt']
    # ssdcm = file['ssdcm']
    # snssi = file['snssi']
    hist_mt = file['hist_mt']
    hist_dc = file['hist_dc']
    hist_ff = file['hist_ff']
    text = file['text']
    bghist_mt = file['bghist_mt']
    bghist_dc = file['bghist_dc']
    bghist_ff = file['bghist_ff']
    file.close()

fig = plot_hists_3ax_with_bg(
    TARGET, NVARS, mt_edges, dc_edges, ff_edges, sn_edges,
    hist_mt, hist_dc, hist_ff, bghist_mt, bghist_dc, bghist_ff,
    ss_cmap=cm.Blues, label = "$\\nu_\mathrm{inner}$")

In [ ]:
# PLOT
xlabel='Mass [M$_\odot$]'
ylabel='SNR'

# build grid from edges
mtgrid, sngrid, = np.meshgrid(mt_edges, sn_edges)

# make figure
fig, axs = plot.figax_double(
    ncols=5, sharex=True, sharey=True, height=4)

# plot histograms
for aa, ax in enumerate(axs):
    im = axs[aa].pcolormesh(mtgrid, sngrid, np.log10(hist_mt[aa]), cmap=magma_r)
    # title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    axs[aa].text(0.01, 0.99, '$\\nu_\mathrm{inner}$=%s' % text[aa], horizontalalignment='left', verticalalignment='top', 
                 transform=axs[aa].transAxes, color='k', fontsize=12)
    # axs[ii].set_facecolor('k')

# set labels
axs[0].set_ylabel(ylabel)
axs[2].set_xlabel(xlabel)
plt.subplots_adjust(wspace=0)
# fig.tight_layout()

# add colorbar
plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.7, 
             orientation='vertical', pad=0.03, label='$\log N$', )

# save
fig.savefig('/Users/emigardiner/GWs/holodeck/output/figures/bigplots/snr_hist'
            + f'/snr_vs_mass_numden_{TARGET}_v{NVARS}_5axs.png', dpi=100)

SNR or DP vs. distance

# GSMF_phi0 5ax

In [ ]:
TARGET = 'gsmf_phi0'
NSKIES = 100
NVARS = 21
# CALC=True

if CALC: 
    calculate_histograms(TARGET, NFREQS, NREALS, NLOUDEST, NSKIES)
else:
    file = np.load('/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata'
            +f'/snr_hist_{TARGET}_v{NVARS}_5axs.npz')
    mt_edges = file['mt_edges']
    sn_edges = file['sn_edges']
    dc_edges = file['dc_edges']
    # ssmtt = file['ssmtt']
    # ssdcm = file['ssdcm']
    # snssi = file['snssi']
    hist_mt = file['hist_mt']
    hist_dc = file['hist_dc']
    hist_ff = file['hist_ff']
    text = file['text']
    bghist_mt = file['bghist_mt']
    bghist_dc = file['bghist_dc']
    bghist_ff = file['bghist_ff']
    file.close()

fig = plot_hists_3ax_with_bg(
    TARGET, NVARS, mt_edges, dc_edges, ff_edges, sn_edges,
    hist_mt, hist_dc, hist_ff, bghist_mt, bghist_dc, bghist_ff,
    ss_cmap=cm.Greens, label = "$\psi_0$")

In [ ]:
# PLOT
xlabel='Mass [M$_\odot$]'
ylabel='SNR'

# build grid from edges
mtgrid, sngrid, = np.meshgrid(mt_edges, sn_edges)

# make figure
fig, axs = plot.figax_double(
    ncols=5, sharex=True, sharey=True, height=4)

# plot histograms
for aa, ax in enumerate(axs):
    im = axs[aa].pcolormesh(mtgrid, sngrid, np.log10(hist_mt[aa]), cmap=magma_r)
    # title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    axs[aa].text(0.01, 0.99, '$\psi_0$=%s' % text[aa], horizontalalignment='left', verticalalignment='top', 
                 transform=axs[aa].transAxes, color='k', fontsize=12)
    # axs[ii].set_facecolor('k')

# set labels
axs[0].set_ylabel(ylabel)
axs[2].set_xlabel(xlabel)
fig.tight_layout()
plt.subplots_adjust(wspace=0)

# add colorbar
plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.7, 
             orientation='vertical', pad=0.03, label='$\log N$', )

# save
fig.savefig('/Users/emigardiner/GWs/holodeck/output/figures/bigplots/snr_hist'
            + f'/snr_vs_mass_numden_{TARGET}_v{NVARS}_5axs.png', dpi=100)

# Gsmf_mchar0_log10 5ax

In [ ]:
TARGET = 'gsmf_mchar0_log10'
NSKIES = 100
NVARS = 21


# CALC=True

if CALC: 
    calculate_histograms(TARGET, NFREQS, NREALS, NLOUDEST, NSKIES)
else:
    file = np.load('/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata'
            +f'/snr_hist_{TARGET}_v{NVARS}_5axs.npz')
    mt_edges = file['mt_edges']
    sn_edges = file['sn_edges']
    dc_edges = file['dc_edges']
    # ssmtt = file['ssmtt']
    # ssdcm = file['ssdcm']
    # snssi = file['snssi']
    hist_mt = file['hist_mt']
    hist_dc = file['hist_dc']
    hist_ff = file['hist_ff']
    text = file['text']
    bghist_mt = file['bghist_mt']
    bghist_dc = file['bghist_dc']
    bghist_ff = file['bghist_ff']
    file.close()


fig = plot_hists_3ax_with_bg(
    TARGET, NVARS, mt_edges, dc_edges, ff_edges, sn_edges,
    hist_mt, hist_dc, hist_ff, bghist_mt, bghist_dc, bghist_ff,
    ss_cmap=cm.Greens, label = "$\log M_{\psi,0}$")

In [ ]:
# PLOT
xlabel='Mass [M$_\odot$]'
ylabel='SNR'

# build grid from edges
mtgrid, sngrid, = np.meshgrid(mt_edges, sn_edges)

# make figure
fig, axs = plot.figax_double(
    ncols=5, sharex=True, sharey=True, height=4)

# plot histograms
for aa, ax in enumerate(axs):
    im = axs[aa].pcolormesh(mtgrid, sngrid, np.log10(hist_mt[aa]), cmap=magma_r)
    # title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    axs[aa].text(0.01, 0.99, '$M_\mathrm{char,0}$=%s' % text[aa], horizontalalignment='left', verticalalignment='top', 
                 transform=axs[aa].transAxes, color='k', fontsize=12)
    # axs[ii].set_facecolor('k')

# set labels
axs[0].set_ylabel(ylabel)
axs[2].set_xlabel(xlabel)
fig.tight_layout()
plt.subplots_adjust(wspace=0)

# add colorbar
plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.7, 
             orientation='vertical', pad=0.03, label='$\log N$', )

# save
fig.savefig('/Users/emigardiner/GWs/holodeck/output/figures/bigplots/snr_hist'
            + f'/snr_vs_mass_numden_{TARGET}_v{NVARS}_5axs.png', dpi=100)

# mmb_mamp_log10 5ax

In [ ]:
TARGET = 'mmb_mamp_log10'
NSKIES = 100
NVARS = 21


# CALC=True

if CALC: 
    calculate_histograms(TARGET, NFREQS, NREALS, NLOUDEST, NSKIES)
else:
    file = np.load('/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata'
            +f'/snr_hist_{TARGET}_v{NVARS}_5axs.npz')
    mt_edges = file['mt_edges']
    sn_edges = file['sn_edges']
    dc_edges = file['dc_edges']
    # ssmtt = file['ssmtt']
    # ssdcm = file['ssdcm']
    # snssi = file['snssi']
    hist_mt = file['hist_mt']
    hist_dc = file['hist_dc']
    hist_ff = file['hist_ff']
    text = file['text']
    bghist_mt = file['bghist_mt']
    bghist_dc = file['bghist_dc']
    bghist_ff = file['bghist_ff']
    file.close()



fig = plot_hists_3ax_with_bg(
    TARGET, NVARS, mt_edges, dc_edges, ff_edges, sn_edges,
    hist_mt, hist_dc, hist_ff, bghist_mt, bghist_dc, bghist_ff,
    ss_cmap=cm.Oranges, label = "$\log\mu$")


In [ ]:
# PLOT
xlabel='Mass [M$_\odot$]'
ylabel='SNR'

# build grid from edges
mtgrid, sngrid, = np.meshgrid(mt_edges, sn_edges)

# make figure
fig, axs = plot.figax_double(
    ncols=5, sharex=True, sharey=True, height=4)

# plot histograms
for aa, ax in enumerate(axs):
    im = axs[aa].pcolormesh(mtgrid, sngrid, np.log10(hist_mt[aa]), cmap=magma_r)
    # title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    axs[aa].text(0.01, 0.99, '$\mu$=%s' % text[aa], horizontalalignment='left', verticalalignment='top', 
                 transform=axs[aa].transAxes, color='k', fontsize=12)
    # axs[ii].set_facecolor('k')

# set labels
axs[0].set_ylabel(ylabel)
axs[2].set_xlabel(xlabel)
fig.tight_layout()
plt.subplots_adjust(wspace=0)

# add colorbar
plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.7, 
             orientation='vertical', pad=0.03, label='$\log N$', )

# save
fig.savefig('/Users/emigardiner/GWs/holodeck/output/figures/bigplots/snr_hist'
            + f'/snr_vs_mass_numden_{TARGET}_v{NVARS}_5axs.png', dpi=100)

# mmb_scatter_dex

In [ ]:
TARGET = 'mmb_scatter_dex'
NSKIES = 100
NVARS = 21


# CALC=True

if CALC: 
    calculate_histograms(TARGET, NFREQS, NREALS, NLOUDEST, NSKIES)
else:
    file = np.load('/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata'
            +f'/snr_hist_{TARGET}_v{NVARS}_5axs.npz')
    mt_edges = file['mt_edges']
    sn_edges = file['sn_edges']
    dc_edges = file['dc_edges']
    # ssmtt = file['ssmtt']
    # ssdcm = file['ssdcm']
    # snssi = file['snssi']
    hist_mt = file['hist_mt']
    hist_dc = file['hist_dc']
    hist_ff = file['hist_ff']
    text = file['text']
    bghist_mt = file['bghist_mt']
    bghist_dc = file['bghist_dc']
    bghist_ff = file['bghist_ff']
    file.close()


fig = plot_hists_3ax_with_bg(
    TARGET, NVARS, mt_edges, dc_edges, ff_edges, sn_edges,
    hist_mt, hist_dc, hist_ff, bghist_mt, bghist_dc, bghist_ff,
    ss_cmap=cm.Oranges, label = "$\epsilon_\mu$")

In [ ]:
# PLOT
xlabel='Mass [M$_\odot$]'
ylabel='SNR'

# build grid from edges
mtgrid, sngrid, = np.meshgrid(mt_edges, sn_edges)

# make figure
fig, axs = plot.figax_double(
    ncols=5, sharex=True, sharey=True, height=4)

# plot histograms
for aa, ax in enumerate(axs):
    im = axs[aa].pcolormesh(mtgrid, sngrid, np.log10(hist_mt[aa]), cmap=magma_r)
    # title = '%s = %.2f' % ('$\psi_0$', params[TARGET])
    axs[aa].text(0.01, 0.99, '$\epsilon_\mu$=%s' % text[aa], horizontalalignment='left', verticalalignment='top', 
                 transform=axs[aa].transAxes, color='k', fontsize=12)
    # axs[ii].set_facecolor('k')

# set labels
axs[0].set_ylabel(ylabel)
axs[2].set_xlabel(xlabel)
fig.tight_layout()
plt.subplots_adjust(wspace=0)

# add colorbar
plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.7, 
             orientation='vertical', pad=0.03, label='$\log N$', )
# save
fig.savefig('/Users/emigardiner/GWs/holodeck/output/figures/bigplots/snr_hist'
            + f'/snr_vs_mass_numden_{TARGET}_v{NVARS}_5axs.png', dpi=100)